In [3]:
'''
Simplex algorithm in the tableau form. The implementation should work as follows:

Take the problem input from a csv which has the following structure:

    The First line has a keyword MIN or MAX followed by a single integer d --- which is the number of variables in the problem. MIN/MAX denote whether the objective is to be minimized or maximized.
    The Second line is the unit cost vector - u a vector of length d.
    The following lines are the constraints --- one line per constraint. Each constraint of the form 'h op g^T.x'  is given in the csv file as "h op g", where 'op' is one of the comparison operators =,<=,>=; 'h' is a scalar value and g is a vector of length d. The constraints may not necessarily be independent or even consistent.
    The vectors u and g are entered in csv on a single line as a sequence (tab or comma separated) values where each 'value' is of the form "x@i" or just "x" --- "x" is any scalar value and x@i says the i-th component of the vector is x, When it is just "x" --- its position in the sequence will be taken as its index. Values at indices not in the sequence (either positional as in "x" or explicit index as in "x@i") are taken to be zero.
    You can use Numpy / JAX

The output of the Simplex implementation must be one of:

INFEASIBLE (the feasible region is empty) --- this wasn't discussed in the class. I want you to extend what was discussed in the class to detect if the given problem is infeasible.

PASS --- the optimum was successfully found

UNBOUNDED --- no optimum exists because the LP is unbounded

In addition if the output is PASS the program should print out the values of the non-zero variables as a dictionary {variable index: value} and the optimum cost at this solution along with the list of redundant constraints that were removed if any.
'''

'\nSimplex algorithm in the tableau form. The implementation should work as follows:\n\nTake the problem input from a csv which has the following structure:\n\n    The First line has a keyword MIN or MAX followed by a single integer d --- which is the number of variables in the problem. MIN/MAX denote whether the objective is to be minimized or maximized.\n    The Second line is the unit cost vector - u a vector of length d.\n    The following lines are the constraints --- one line per constraint. Each constraint of the form \'h op g^T.x\'  is given in the csv file as "h op g", where \'op\' is one of the comparison operators =,<=,>=; \'h\' is a scalar value and g is a vector of length d. The constraints may not necessarily be independent or even consistent.\n    The vectors u and g are entered in csv on a single line as a sequence (tab or comma separated) values where each \'value\' is of the form "x@i" or just "x" --- "x" is any scalar value and x@i says the i-th component of the vect

In [1]:
'''
TODO:-
=> Implement the input reading from csv
=> Remove dependent constraints
=> Handle inconsistent constraints
'''

'\nTODO:-\n=> Implement the simplex algorithm for Min.\n=> Implement for Max. [u->-u in SimplexSolver]\n=> Implement the input reading from csv\n=> Remove dependent constraints\n=> Handle inconsistent constraints\n\nPossible bugs:-\n=> Keep track of x[i] getting negative\n'

In [2]:
import numpy as np

In [3]:
# # Represent numpy arrays as fractions
# from fractions import Fraction
# np.set_printoptions(formatter={'all':lambda x: str(Fraction(x).limit_denominator())})

In [70]:
class SimplexSolver:
    def __init__(self, type, d, u, constraints):
        self.type = type
        self.d_old = d
        self.d = d
        self.u = u
        if type == 'MAX':
            self.u = -self.u
        self.constraints = constraints
        self.addSlackVariables()
        self.G = np.array([constraint[2] for constraint in self.constraints])
        self.c = self.G.shape[0]
        self.h_tilde = np.array([constraint[0] for constraint in self.constraints])
        self.numAuxVars = self.c - self.d

    def addSlackVariables(self):
        for i in range(len(self.constraints)):
            if self.constraints[i][1] in ['<=', '>=']:
                self.d += 1
                self.u = np.append(self.u, 0)

        # Add slack variables to the constraints according to the mask
        slack_count = 0
        for i in range(len(self.constraints)):
            if self.constraints[i][1] == '>=':
                slack_count += 1
                self.constraints[i][1] = '='                    # Convert the constraint to equality
                g_new = np.zeros(self.d)
                g_new[:self.d_old] = self.constraints[i][2]     # Copy the original constraints

                slack_index = self.d_old + slack_count - 1

                g_new[slack_index] = 1                          # Add slack variable

                self.constraints[i][2] = g_new
            elif self.constraints[i][1] == '<=':
                slack_count += 1
                self.constraints[i][1] = '='                    # Convert the constraint to equality
                g_new = np.zeros(self.d)
                g_new[:self.d_old] = self.constraints[i][2]

                slack_index = self.d_old + slack_count - 1

                g_new[slack_index] = -1

                self.constraints[i][2] = g_new
            else:
                # No need to add slack variable for equality
                self.constraints[i][2] = np.concatenate((self.constraints[i][2], np.zeros(self.d - self.d_old)))

    def getInitialLPP(self, ):
        # Get the auxiliary LPP for finding initial BFS
        # Minimize y1 + y2 + ... + yc subject to
        # [G, I] * [x, y].T = h~
        
        # Construct G' = [G, I]
        G1 = np.concatenate((self.G, np.eye(self.c)), axis=1)
        u1 = np.concatenate((np.zeros((self.d,)), np.ones((self.c,))))    # (d+c,)
        h_tilde1 = self.h_tilde                                    # (c,)
        return G1, u1, h_tilde1
    
    def getTableau(self, G, u, h_tilde):
        # Construct the tableau for the given LPP
        # Assume G: (c,d), u: (d,), h_tilde: (c,)
        uT = u.reshape(1, -1)
        tableau = np.concatenate((G, uT), axis=0)
        h_tilde = h_tilde.reshape(-1, 1)
        h_tilde = np.concatenate((h_tilde, np.zeros((1, 1))), axis=0)
        tableau = np.concatenate((tableau, h_tilde), axis=1)
        return tableau
    
    def getCanonicalForm(self, tableau, basis):
        '''
        Canonical form : A system Ax = b is said to be in canonical form if among
        the n variables there are m variables with the property that each appears in
        only one equation, and its coefficient in that equation is unity.
        [An introduction to optimization definition 16.5]

        To convert into canonical form, we'll make the columns corresponding
        to the basic variables in last row to 0 using row operations.
        '''
        # TODO : Can it be optimized using numpy?
        # Reduce the elements of the last row of the basic columns to 0
        for i in range(len(basis)):
            if basis[i] != -1:
                tableau[-1, :] -= tableau[basis[i], :] * tableau[-1, i]
        return tableau
    
    def getLeavingVar(self, tableau, entering_var, basis):
        basisMask = (basis != -1)

        # Handle precision errors
        closeZero = np.isclose(tableau, 0)
        tableau[closeZero] = 0

        # Handle unbounded LPP
        if np.all(tableau[:-1, entering_var] <= 0):
            return None
        else:
            # We need to consider only the non basic columns, so mask the basic columns
            basisMask = basisMask.reshape(1, -1)
            tableau = tableau.copy()
            # Mask will be applied on just (c,d) part of the tableau
            tableau[:-1, :-1] = tableau[:-1, :-1] * (1 - basisMask)

            # Need to find argmin(hi / nie) : nie > 0
            nie = tableau[:-1, entering_var]
            hi = tableau[:-1, -1]

            # We only need to consider the rows where nie > 0
            nMask = (nie <= 0)
            nie[nMask] = 1

            ratio = hi / nie
            ratio[nMask] = np.inf           # Set the ratio to infinity if nie = 0

            leaving_var = np.argmin(ratio)  # Returns the first occurence of the minimum value
            return leaving_var              # Note: This is the index of the leaving variable in the basis, not tableau

    def getEnteringVar(self, tableau):
        # Get the entering variable from the given tableau

        # Handle precision errors
        closeZero = np.isclose(tableau, 0)
        tableau[closeZero] = 0

        # First check if the tableau is optimal
        if np.all(tableau[-1, :-1] >= 0):
            return None
        else:
            # Get the entering variable (assuming canonical form so we're just checking entries of (u_n - N.T * u_B))
            # Find the first negative element in the last row
            entering_var = np.where(tableau[-1, :-1] < 0)[0][0]
            return entering_var

    def swapBasisVar(self, tableau, entering_var, leaving_var, basis):
        # Get position of leaving_var in tableau
        leaving_var_tab = np.where(basis == leaving_var)[0][0]
        
        # Entering_var is the index in the tableau itself, so no need to update it

        # Update the tableau
        tableau[leaving_var, :] /= tableau[leaving_var, entering_var]
        reduce_col = tableau[:, entering_var].copy()
        reduce_col[leaving_var] = 0      # Exclude this row to prevent basis column from becoming 0
        tableau -= np.matmul(reduce_col.reshape(-1, 1), tableau[leaving_var, :].reshape(1,-1))

        # Swap the entering and leaving variables in the basis
        basis[leaving_var_tab] = -1
        basis[entering_var] = leaving_var

        return tableau, basis
    
    def getOptimalSolution(self, tableau, basis):
        # Simplex algorithm to get the optimal solution
        entering_var = self.getEnteringVar(tableau)
        while entering_var is not None:
            leaving_var = self.getLeavingVar(tableau, entering_var, basis)
            if leaving_var is None:
                return None, None
            tableau, basis = self.swapBasisVar(tableau, entering_var, leaving_var, basis)
            entering_var = self.getEnteringVar(tableau)
            print(tableau)

        return tableau, basis

    def solve(self, ):
        # First get initial BFS by solving the auxiliary LPP
        G1, u1, h_tilde1 = self.getInitialLPP()
        tableau1 = self.getTableau(G1, u1, h_tilde1)
        # print(tableau1)

        # Basis as a binary mask on all the variables
        basis = np.concatenate((-1 * np.ones((self.d,), dtype=int), np.arange(self.c, dtype=int)))      # To keep track of ith basic variable
        tableau = self.getCanonicalForm(tableau1, basis)
        print(tableau)
        
        # Get the optimal solution
        tableau, basis = self.getOptimalSolution(tableau, basis)

        # TODO : Check if the LPP is unbounded
        # if (tableau, basis) == (None, None):      # TODO: Check the error here
        #     return 'UNBOUNDED'
        
        # Solution of the auxiliary LPP is the initial BFS
        auxSol = tableau[:-1, -1]
        print("Auxiliary solution: ")
        print(auxSol)

        # Apply simplex algorithm on the initial BFS to get the optimal solution
        # Remove the columns corresponding to the auxiliary variables

        # Remove last c columns and add the last column to the end
        tableau = np.concatenate((tableau[:-1, :-(self.c+1)], tableau[:-1, -1].reshape(-1,1)), axis=1)
        u_0 = np.concatenate((self.u.reshape(1,-1), np.zeros((1,1))), axis=1)
        tableau = np.concatenate((tableau, u_0), axis=0)
        
        # New basis
        basis = basis[:-self.c]
        print(basis.shape)

        # Get canonical form
        tableau = self.getCanonicalForm(tableau, basis)
        print(tableau)

        # Optimize the new tableau
        tableau, basis = self.getOptimalSolution(tableau, basis)
        print(tableau)

In [122]:
type = "MIN"
d = 3
u = np.array([1, 2, 3])
constraints = [
    [1, "<=", np.array([1, 2, 3])],
    [2, ">=", np.array([-2, 4, 7])],
    [3, "=", np.array([0, 1, 2])],
    [4, "<=", np.array([1, 1, 1])]
]


type = "MIN"
d = 2
u = np.array([2, 3])
constraints = [
    [12, "<=", np.array([4, 2])],
    [6, "<=", np.array([1, 4])],
]

type = "MAX"
d = 3
u = np.array([4, 1, 4])
constraints = [
    [2, ">=", np.array([2,1,1])],
    [4, ">=", np.array([1,2,3])],
    [8, ">=", np.array([2,2,1])],
]

type="MIN"
d=4
u=np.array([1,1,0,0])
constraints=[
    [8, "=", np.array([1,1,0,1])],
    [10, "=", np.array([2, 1, 1, 0])],
]

type = "MIN"
d=2
u = np.array([-40, -30])
constraints = [
    [12, ">=", np.array([1, 1])],
    [16, ">=", np.array([2, 1])],
]

type="MIN"
d=2
u=np.array([4,1])
constraints = [
    [3, ">=", np.array([1, 2])],
    [6, "<=", np.array([4, 3])],
    [3, "=", np.array([3, 1])],
]

type="MIN"
d=4
u=np.array([1,-1,2,-2])
constraints = [
    [1, "=", np.array([1,-1,1,-2])],
    [4, "=", np.array([2,-2,1,-1])],
]

type="MAX"
d=2
u=np.array([3,5])
constraints = [
    [4, ">=", np.array([1,1])],
    [8, "<=", np.array([5,3])],
]

type="MIN"
d=4
u=np.array([2,-1,-1,0])       # works
constraints = [
    [4, "=", np.array([3,1,0,1])],
    [5, "=", np.array([6,2,1,1])],
]

type="MAX"
d=3
u=np.array([1,1,3])    # works
constraints = [
    [1, "=", np.array([1,0,1])],
    [2, "=", np.array([0,1,1])],
]

type="MAX"
d=2
u=np.array([2,1])       # works
constraints = [
    [5, ">=", np.array([1,0])],
    [7, ">=", np.array([0,1])],
]

type="MIN"
d=2
u=np.array([1,1])       # works
constraints = [
    [3, "<=", np.array([1,2])],
    [3, "<=", np.array([2,1])],
]

type="MAX"
d=2
u=np.array([-4,-3])       # works
constraints = [
    [11, "<=", np.array([5,1])],
    [-8, ">=", np.array([-2,-1])],
    [7. , "<=", np.array([1,2])],
]

type="MAX"
d=4
u=np.array([6,4,7,5])       # works
constraints = [
    [20, ">=", np.array([1,2,1,2])],
    [100, '>=', np.array([6,5,3,2])],
    [75, ">=", np.array([3,4,9,12])],
]

In [118]:
simplex = SimplexSolver(type, d, u, constraints)

In [119]:
simplex.solve()

[[   1.    2.    1.    2.    1.    0.    0.    1.    0.    0.   20.]
 [   6.    5.    3.    2.    0.    1.    0.    0.    1.    0.  100.]
 [   3.    4.    9.   12.    0.    0.    1.    0.    0.    1.   75.]
 [ -10.  -11.  -13.  -16.   -1.   -1.   -1.    0.    0.    0. -195.]]
[[  0.           1.16666667   0.5          1.66666667   1.
   -0.16666667   0.           1.          -0.16666667   0.
    3.33333333]
 [  1.           0.83333333   0.5          0.33333333   0.
    0.16666667   0.           0.           0.16666667   0.
   16.66666667]
 [  0.           1.5          7.5         11.           0.
   -0.5          1.           0.          -0.5          1.
   25.        ]
 [  0.          -2.66666667  -8.         -12.66666667  -1.
    0.66666667  -1.           0.           1.66666667   0.
  -28.33333333]]
[[  0.           1.           0.42857143   1.42857143   0.85714286
   -0.14285714   0.           0.85714286  -0.14285714   0.
    2.85714286]
 [  1.           0.           0.14285714  -0

In [120]:
# Create a function to transform from constraints to G for linprog
def constraintsToG(constraints):
    G = np.zeros((len(constraints), d))
    for i in range(len(constraints)):
        print(constraints[i])
        if constraints[i][1] == '<=':
            G[i] = -1 * constraints[i][2][:d]
        else:
            G[i] = constraints[i][2][:d]
    return G

def constraintsToH(constraints):
    h = np.zeros((len(constraints),))
    for i in range(len(constraints)):
        if constraints[i][1] == '<=':
            h[i] = -1 * constraints[i][0]
        else:
            h[i] = constraints[i][0]
    return h

In [123]:
# Solve the LPP using linprog
from scipy.optimize import linprog
c = simplex.u[:simplex.d_old]
print(c)
G = constraintsToG(constraints)
print(G)
h = constraintsToH(constraints)       # Form h >= g^T.x
print(h)
# Print with steps
res = linprog(c, A_ub=G, b_ub=h, method='simplex', options={'disp': True})
print(res)

[-6 -4 -7 -5]
[20, '>=', array([1, 2, 1, 2])]
[100, '>=', array([6, 5, 3, 2])]
[75, '>=', array([ 3,  4,  9, 12])]
[[ 1.  2.  1.  2.]
 [ 6.  5.  3.  2.]
 [ 3.  4.  9. 12.]]
[ 20. 100.  75.]
Optimization terminated successfully.
         Current function value: -113.333333 
         Iterations: 4
 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: -113.33333333333333
       x: [ 1.500e+01  0.000e+00  3.333e+00  0.000e+00]
     nit: 4


/tmp/ipykernel_4919/399770180.py:10: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res = linprog(c, A_ub=G, b_ub=h, method='simplex', options={'disp': True})


In [110]:
# Solve the LPP using linprog
from scipy.optimize import linprog
c = simplex.u[:simplex.d_old]
print(c)
G = constraintsToG(constraints)
print(G)
h = constraintsToH(constraints)       # Form h >= g^T.x
print(h)
# Print with steps
res = linprog(c, A_eq=G, b_eq=h, method='simplex', options={'disp': True})
print(res)

[4 3]
[11, '=', array([ 5.,  1., -1.,  0.,  0.])]
[-8, '=', array([-2., -1.,  0.,  1.,  0.])]
[7.0, '=', array([ 1.,  2.,  0.,  0., -1.])]
[[ 5.  1.]
 [-2. -1.]
 [ 1.  2.]]
[11. -8.  7.]
There is a linear combination of rows of A_eq that results in zero, suggesting a redundant constraint. However the same linear combination of b_eq is nonzero, suggesting that the constraints conflict and the problem is infeasible.
         Iterations: 0
 message: There is a linear combination of rows of A_eq that results in zero, suggesting a redundant constraint. However the same linear combination of b_eq is nonzero, suggesting that the constraints conflict and the problem is infeasible.
 success: False
  status: 2
     fun: 0.0
       x: [ 0.000e+00  0.000e+00]
     nit: 0


/tmp/ipykernel_4919/4065296701.py:10: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res = linprog(c, A_eq=G, b_eq=h, method='simplex', options={'disp': True})
/tmp/ipykernel_4919/4065296701.py:10: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  res = linprog(c, A_eq=G, b_eq=h, method='simplex', options={'disp': True})
